# [모듈 8.5] 전체 모델 빌딩 파이프라인

## 0. 기본 세이지 메이커 정보 및 기본 변수 로딩

In [1]:
import boto3
import sagemaker
import pandas as pd

region = boto3.Session().region_name
sagemaker_session = sagemaker.session.Session()
role = sagemaker.get_execution_role()

%store -r 

## 1. 모델 빌딩 파이프라인 변수 생성



In [2]:
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
)

processing_instance_count = ParameterInteger(
    name="ProcessingInstanceCount",
    default_value=1
)
processing_instance_type = ParameterString(
    name="ProcessingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_type = ParameterString(
    name="TrainingInstanceType",
    default_value="ml.m5.xlarge"
)

training_instance_count = ParameterInteger(
    name="TrainingInstanceCount",
    default_value=1
)


input_data = ParameterString(
    name="InputData",
    default_value=input_data_uri,
)


## 2.전처리 스텝 단계 정의
- input_data_uri 입력 데이타를 대상으로 전처리를 수행 합니다.

In [3]:
from sagemaker.sklearn.processing import SKLearnProcessor

split_rate = 0.2
framework_version = "0.23-1"

sklearn_processor = SKLearnProcessor(
    framework_version=framework_version,
    instance_type=processing_instance_type,
    instance_count=processing_instance_count,
    base_job_name="sklearn-fraud-process",
    role=role,
)
print("input_data: \n", input_data)

input_data: 
 s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step/input


In [4]:
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.workflow.steps import ProcessingStep
    
step_process = ProcessingStep(
    name="FraudScratchProcess",
    processor=sklearn_processor,
    inputs=[
#         ProcessingInput(source=input_data_uri,destination='/opt/ml/processing/input'),
        ProcessingInput(source=input_data, destination='/opt/ml/processing/input'),        
         ],
    outputs=[ProcessingOutput(output_name="train",
                              source='/opt/ml/processing/output/train'),
             ProcessingOutput(output_name="test",
                              source='/opt/ml/processing/output/test')],
    job_arguments=["--split_rate", f"{split_rate}"],        
    code= 'src/preprocessing.py',
)


## 3.모델 학습을 위한 학습단계 정의 



### 기본 훈련 변수 및 하이퍼파라미터 설정

In [5]:
from sagemaker.xgboost.estimator import XGBoost

bucket = sagemaker_session.default_bucket()
prefix = 'fraud2train'

estimator_output_path = f's3://{bucket}/{prefix}/training_jobs'
#train_instance_count = 1

hyperparameters = {
       "scale_pos_weight" : "29",        
        "max_depth": "3",
        "eta": "0.2",
        "objective": "binary:logistic",
        "num_round": "100",
}

In [6]:
xgb_train = XGBoost(
    entry_point = "xgboost_starter_script.py",
    source_dir = "src",
    output_path = estimator_output_path,
    code_location = estimator_output_path,
    hyperparameters = hyperparameters,
    role = role,
    instance_count = training_instance_count,
    instance_type = training_instance_type,
    framework_version = "1.0-1")

훈련의 입력이 이전 전처리의 결과가 제공됩니다.
- `step_process.properties.ProcessingOutputConfig.Outputs["train"].S3Output.S3Uri`

In [7]:
from sagemaker.inputs import TrainingInput
from sagemaker.workflow.steps import TrainingStep


step_train = TrainingStep(
    name="FraudScratchTrain",
    estimator=xgb_train,
    inputs={
        "train": TrainingInput(
            s3_data=step_process.properties.ProcessingOutputConfig.Outputs[
                "train"
            ].S3Output.S3Uri,
            # s3_data= train_preproc_dir_artifact,            
            content_type="text/csv"
        ),
    },
)

## 4.세이지 메이커 모델 스텝 생성
- 아래 두 파리미터의 입력이 이전 스텝의 결과가 제공됩니다.
    - image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    - model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,



In [8]:
from sagemaker.model import Model
    
model = Model(
    image_uri= step_train.properties.AlgorithmSpecification.TrainingImage,
    model_data= step_train.properties.ModelArtifacts.S3ModelArtifacts,
    sagemaker_session=sagemaker_session,
    role=role,
)

In [9]:
from sagemaker.inputs import CreateModelInput
from sagemaker.workflow.steps import CreateModelStep


inputs = CreateModelInput(
    instance_type="ml.m5.large",
    # accelerator_type="ml.eia1.medium",
)
step_create_model = CreateModelStep(
    name="FraudScratchModel",
    model=model,
    inputs=inputs,
)

## 5.실시간 엔드 포인트 배포 스텝 생성
- endpoint config 생성시에, 이전 단계의 모델 결과를 제공합니다.
    - "--model_name", step_create_model.properties.ModelName, 

In [10]:
from datetime import datetime
suffix = datetime.now().microsecond

In [11]:

local_deploy_code_path = 'src/deploy_model.py'
s3_deploy_code_path = f"s3://{default_bucket}/{project_prefix}/code"
s3_deploy_code_uri = sagemaker.s3.S3Uploader.upload(
    local_path=local_deploy_code_path, 
    desired_s3_uri=s3_deploy_code_path,
)
print("s3_deploy_code_uri: ", s3_deploy_code_uri)

all_pipeline_endpoint_name = 'all-pipeline-endpoint-' + str(suffix)
endpoint_instance_type = "ml.m5.xlarge"

s3_deploy_code_uri:  s3://sagemaker-ap-northeast-2-057716757052/sagemaker-pipeline-step-by-step/code/deploy_model.py


In [12]:
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.workflow.steps import ProcessingStep

deploy_model_processor = SKLearnProcessor(
    framework_version='0.23-1',
    role= role,
    instance_type="ml.t3.medium",
    instance_count=1,
    base_job_name='fraud-scratch-deploy-model',
    sagemaker_session=sagemaker_session)


step_deploy = ProcessingStep(
    name='DeployModel',
    processor=deploy_model_processor,
    job_arguments=[
        "--model_name", step_create_model.properties.ModelName, 
        "--region", region,
        "--endpoint_instance_type", endpoint_instance_type,
        "--endpoint_name", all_pipeline_endpoint_name
    ],
    code=s3_deploy_code_uri)

## 6.모델 빌딩 파이프라인 정의 및 실행
위에서 정의한 아래의 4개의 스텝으로 파이프라인 정의를 합니다.
-     steps=[step_process, step_train, step_create_model, step_deploy],
- 아래는 약 20분 정도 소요 됩니다.

In [13]:
from sagemaker.workflow.pipeline import Pipeline

project_prefix = 'sagemaker-pipeline-step-by-step'

pipeline_name = project_prefix
pipeline = Pipeline(
    name=pipeline_name,
    parameters=[
        processing_instance_type, 
        processing_instance_count,
        training_instance_type,        
        training_instance_count,                
        input_data,
    ],
    steps=[step_process, step_train, step_create_model, step_deploy],
)

In [14]:
import json

definition = json.loads(pipeline.definition())
# definition

### 파이프라인을 SageMaker에 제출하고 실행하기 


In [15]:
pipeline.upsert(role_arn=role)

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step',
 'ResponseMetadata': {'RequestId': '2906d88c-a711-437a-b2cd-7e681d621838',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2906d88c-a711-437a-b2cd-7e681d621838',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '104',
   'date': 'Sun, 27 Jun 2021 14:06:51 GMT'},
  'RetryAttempts': 0}}

디폴트값을 이용하여 파이프라인을 샐행합니다. 

In [16]:
execution = pipeline.start()

### 파이프라인 운영: 파이프라인 대기 및 실행상태 확인

워크플로우의 실행상황을 살펴봅니다. 

In [17]:
execution.describe()

{'PipelineArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step',
 'PipelineExecutionArn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:pipeline/sagemaker-pipeline-step-by-step/execution/eg0revhx2vbq',
 'PipelineExecutionDisplayName': 'execution-1624802812308',
 'PipelineExecutionStatus': 'Executing',
 'CreationTime': datetime.datetime(2021, 6, 27, 14, 6, 52, 237000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2021, 6, 27, 14, 6, 52, 237000, tzinfo=tzlocal()),
 'CreatedBy': {},
 'LastModifiedBy': {},
 'ResponseMetadata': {'RequestId': 'cfdf2b0a-1704-4b3b-a209-66293dbfe91f',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'cfdf2b0a-1704-4b3b-a209-66293dbfe91f',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '437',
   'date': 'Sun, 27 Jun 2021 14:06:51 GMT'},
  'RetryAttempts': 0}}

In [ ]:

execution.wait()

실행이 완료될 때까지 기다립니다.

실행된 단계들을 리스트업합니다. 파이프라인의 단계실행 서비스에 의해 시작되거나 완료된 단계를 보여줍니다.

In [22]:
execution.list_steps()

[{'StepName': 'DeployModel',
  'StartTime': datetime.datetime(2021, 6, 27, 14, 13, 48, 443000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 6, 27, 14, 30, 0, 672000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'ProcessingJob': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:processing-job/pipelines-eg0revhx2vbq-deploymodel-elzqyscld6'}}},
 {'StepName': 'FraudScratchModel',
  'StartTime': datetime.datetime(2021, 6, 27, 14, 13, 46, 838000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 6, 27, 14, 13, 47, 927000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'Model': {'Arn': 'arn:aws:sagemaker:ap-northeast-2:057716757052:model/pipelines-eg0revhx2vbq-fraudscratchmodel-qxvajnttdu'}}},
 {'StepName': 'FraudScratchTrain',
  'StartTime': datetime.datetime(2021, 6, 27, 14, 10, 52, 311000, tzinfo=tzlocal()),
  'EndTime': datetime.datetime(2021, 6, 27, 14, 13, 46, 436000, tzinfo=tzlocal()),
  'StepStatus': 'Succeeded',
  'Metadata': {'

#### 아티펙트 경로 추출

In [63]:
def get_proc_artifact(execution, client, kind=0):
    '''
    kind: 0 --> train
    kind: 2 --> test
    '''
    response = execution.list_steps()

    proc_arn = response[-1]['Metadata']['ProcessingJob']['Arn'] # index -1은 가장 처음 실행 step
    #proc_arn = response[-1]['Metadata']
    # print("proc_arn: ", proc_arn)
    proc_job_name = proc_arn.split('/')[-1]
    print("proc_job_name: ", proc_job_name)
    
    response = client.describe_processing_job(ProcessingJobName = proc_job_name)
    test_preprocessed_file = response['ProcessingOutputConfig']['Outputs'][kind]['S3Output']['S3Uri'] # index 1: test 파일    
    print("test_preprocessed_file: \n ", test_preprocessed_file)
    
    return test_preprocessed_file

import boto3
client = boto3.client("sagemaker")

test_preproc_dir_artifact = get_proc_artifact(execution, client, kind=1 )

#print("test_preproc__dir_artifact: ", test_preproc_dir_artifact)



proc_job_name:  pipelines-eg0revhx2vbq-fraudscratchprocess-9uu76wleac
test_preprocessed_file: 
  s3://sagemaker-ap-northeast-2-057716757052/sklearn-fraud-process-2021-06-27-14-06-50-834/output/test


## 변수 저장

In [64]:
%store test_preproc_dir_artifact
%store all_pipeline_endpoint_name

Stored 'test_preproc_dir_artifact' (str)
Stored 'all_pipeline_endpoint_name' (str)
